In [1]:
# Defect Detection 

In [2]:
# Import Necessary libraries
import tensorflow as tf
from tensorflow.keras import layers
import os
import numpy as np
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [3]:
# Test image and Train image path
train_path = 'archive/casting_data/casting_data/train'
test_path = 'archive/casting_data/casting_data/test'

# Here load the image , take label from it, resize image
train = tf.keras.utils.image_dataset_from_directory(
    train_path,
    label_mode='int',
    image_size=(128, 128),
    batch_size=32,
    shuffle=True
)
test = tf.keras.utils.image_dataset_from_directory(
    test_path,
    label_mode='int',
    image_size=(128, 128),
    batch_size=32,
    shuffle=False
)

Found 6633 files belonging to 2 classes.
Found 715 files belonging to 2 classes.


In [4]:
# here normalize image and augumented
augment_normalise = tf.keras.Sequential([
    layers.Rescaling(1./255),
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
])

In [5]:
# Here using pretrained MobileNetV2 a CNN model using 
base_model = tf.keras.applications.MobileNetV2(input_shape=(128, 128, 3), include_top=False, weights='imagenet')
base_model.trainable = False

model = tf.keras.Sequential([
    augment_normalise,
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(2, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ sequential (Sequential)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ mobilenetv2_1.00_128 (Functional)    │ (None, 4, 4, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ ?                           │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,257,984 (8.61 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 2,257,984 (8.61 MB)

In [6]:
# training the model with our images
history = model.fit(train, validation_data=test, epochs=10, batch_size=32)

Epoch 1/10
208/208 ━━━━━━━━━━━━━━━━━━━━ 43s 188ms/step - accuracy: 0.8276 - loss: 0.3578 - val_accuracy: 0.9706 - val_loss: 0.0925
Epoch 2/10
208/208 ━━━━━━━━━━━━━━━━━━━━ 39s 187ms/step - accuracy: 0.9524 - loss: 0.1247 - val_accuracy: 0.8965 - val_loss: 0.2266
Epoch 3/10
208/208 ━━━━━━━━━━━━━━━━━━━━ 41s 198ms/step - accuracy: 0.9708 - loss: 0.0823 - val_accuracy: 0.9762 - val_loss: 0.0674
Epoch 4/10
208/208 ━━━━━━━━━━━━━━━━━━━━ 39s 188ms/step - accuracy: 0.9771 - loss: 0.0788 - val_accuracy: 0.9804 - val_loss: 0.0576
Epoch 5/10
208/208 ━━━━━━━━━━━━━━━━━━━━ 40s 190ms/step - accuracy: 0.9742 - loss: 0.0701 - val_accuracy: 0.9790 - val_loss: 0.0569
Epoch 6/10
208/208 ━━━━━━━━━━━━━━━━━━━━ 39s 189ms/step - accuracy: 0.9795 - loss: 0.0656 - val_accuracy: 0.9804 - val_loss: 0.0533
Epoch 7/10
208/208 ━━━━━━━━━━━━━━━━━━━━ 39s 186ms/step - accuracy: 0.9840 - loss: 0.0499 - val_accuracy: 0.9846 - val_loss: 0.0434
Epoch 8/10
208/208 ━━━━━━━━━━━━━━━━━━━━ 39s 187ms/step - accuracy: 0.9731 - loss: 0

In [7]:
#Testing and Metrics
y_true = []
y_pred = []

for images, labels in test:
    preds = model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(preds, axis=1))

f1 = f1_score(y_true, y_pred)
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)

print("F1 Score:", f1)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 202ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
F1 Score: 0.9792843691148776
Accuracy: 0.9846153846153847
Precision: 0.966542750929368
Recall: 0.992366412213

In [8]:
# Save the model
model.save("defect_model.h5")

In [9]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_true, y_pred)
print(cm)

[[444   9]
 [  2 260]]
